<a href="https://colab.research.google.com/github/mgaiduk/mgaiduk/blob/main/grasp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.metrics import f1_score

tf.get_logger().setLevel('ERROR')
print(tf.__version__)
tf.config.list_physical_devices()

2.13.1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
def load_data(path, is_val):
  df = pd.read_csv(path, sep="\t", names=["topicId", "topic_name", "sent1", "sent2", "label", "sent1_tag", "sent2_tag"])
  def label_remap_train(label_str):
    numbers = label_str.strip("()").split(", ")
    number = int(numbers[0])
    if number >= 3:
      return 1
    if number <= 1:
      return 0
    return 2 # to be filtered out
  def label_remap_val(label_str):
    number = int(label_str)
    if number >= 4:
      return 1
    if number <= 2:
      return 0
    return 2 # to be filtered out
  if is_val:
    label_remap = label_remap_val
  else:
    label_remap = label_remap_train
  df["label_remapped"] = df["label"].apply(label_remap)
  if not is_val:
    df = df[df["label_remapped"] != 2]
  s1 = tf.constant(df["sent1"])
  s2 = tf.constant(df["sent2"])
  l = tf.constant(df["label_remapped"])
  ds = tf.data.Dataset.from_tensor_slices(({"s1": s1, "s2": s2}, l)).batch(32)
  return ds
train_ds = load_data("/content/drive/MyDrive/Colab Notebooks/data/train.data", is_val=False)
dev_ds = load_data("/content/drive/MyDrive/Colab Notebooks/data/dev.data", is_val=False)
val_ds = load_data("/content/drive/MyDrive/Colab Notebooks/data/test.data", is_val=True)
for r in train_ds:
  break
r

({'s1': <tf.Tensor: shape=(32,), dtype=string, numpy=
  array([b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'EJ Manuel the 1st QB to go in this draft',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',
         b'So EJ Manuel is the 1st QB taken',


In [124]:
#bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_model_name = "bert_en_uncased_L-12_H-768_A-12"

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [125]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [126]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.92169887 -0.39353466 -0.53931785  0.6825625   0.43848515 -0.14021172
  0.87747127  0.26043358 -0.6311301  -0.9999658  -0.26320052  0.85105324]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[ 0.19451568  0.25141704  0.19075024 ... -0.24845096  0.38568568
   0.13291004]
 [-0.5947861  -0.3942036   0.25245717 ... -0.7694675   1.1564167
   0.32475695]
 [ 0.00641518 -0.15766475  0.5461027  ... -0.17451018  0.6028964
   0.42672247]
 ...
 [ 0.21948306 -0.20927139  0.5386825  ...  0.24693541  0.1825099
  -0.44427088]
 [ 0.01080238 -0.44553143  0.35990992 ...  0.3172276   0.2356278
  -0.630706  ]
 [ 0.2932116  -0.10581895  0.6114753  ...  0.2074579   0.14494637
  -0.35353395]]


In [127]:
preprocessor = hub.load(tfhub_handle_preprocess)

In [128]:
def build_classifier_model():
  s1_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='s1_input')
  s2_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='s2_input')
  tokenize = hub.KerasLayer(preprocessor.tokenize)
  tokenized_inputs = [tokenize(s1_input), tokenize(s2_input)]
  seq_length = 128  # Your choice here.
  bert_pack_inputs = hub.KerasLayer(
    preprocessor.bert_pack_inputs,
    arguments=dict(seq_length=seq_length))  # Optional argument.
  encoder_inputs = bert_pack_inputs(tokenized_inputs)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='dense1')(net)
  return tf.keras.Model({"s1": s1_input, "s2": s2_input}, net)
classifier_model = build_classifier_model()
classifier_model(r[0])

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[-1.1033115],
       [-1.1097867],
       [-1.1048901],
       [-1.1059434],
       [-1.0818659],
       [-1.1315881],
       [-1.1048632],
       [-1.1178727],
       [-1.142633 ],
       [-1.1156156],
       [-1.1067151],
       [-1.1034857],
       [-1.1012384],
       [-1.0639527],
       [-1.1053207],
       [-1.0997252],
       [-1.094215 ],
       [-1.0960128],
       [-1.0879775],
       [-1.1355648],
       [-1.140933 ],
       [-1.125597 ],
       [-1.1376677],
       [-1.1276983],
       [-1.1576236],
       [-1.1156662],
       [-1.1271447],
       [-1.1810457],
       [-1.1672139],
       [-1.1405157],
       [-1.1558399],
       [-1.1649184]], dtype=float32)>

In [129]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [130]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [131]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=dev_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/5
361/361 [==============================] - 421s 1s/step - loss: 0.5408 - binary_accuracy: 0.7355 - val_loss: 0.6010 - val_binary_accuracy: 0.7045
Epoch 2/5
361/361 [==============================] - 399s 1s/step - loss: 0.3639 - binary_accuracy: 0.8509 - val_loss: 0.6789 - val_binary_accuracy: 0.7310
Epoch 3/5
361/361 [==============================] - 400s 1s/step - loss: 0.2329 - binary_accuracy: 0.9127 - val_loss: 0.6893 - val_binary_accuracy: 0.7803
Epoch 4/5
361/361 [==============================] - 400s 1s/step - loss: 0.1255 - binary_accuracy: 0.9552 - val_loss: 1.0114 - val_binary_accuracy: 0.7805
Epoch 5/5
361/361 [==============================] - 432s 1s/step - loss: 0.0703 - binary_accuracy: 0.9760 - val_loss: 1.1575 - val_binary_accuracy: 0.7774


In [136]:
classifier_model.save("/content/drive/MyDrive/Colab Notebooks/data/model.wad")

In [135]:
# getting predictions
all_preds = []
threshold = 0.1
with open("/content/drive/MyDrive/Colab Notebooks/data/output.tsv", "w") as file:
  for batch in val_ds:
    preds = classifier_model(batch[0])
    for row in preds:
      pred = tf.sigmoid(row[0]).numpy()
      all_preds.append(pred)
      if pred >= threshold:
        label = "true"
      else:
        label = "false"
      print(label + "\t{:.4f}".format(pred), file=file)
all_preds[:5]

[0.0003251722, 0.00018419913, 0.0003987643, 0.0002744785, 0.00022812486]

In [59]:
# loading test labels to calculate f1 score
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/test.label", sep="\t", names=["label", "score"])
def remap(label_str):
    if label_str == "true":
        return 1
    if label_str == "false":
        return 0
    return 2 # to be filtered out later
df["label_remapped"] = df["label"].apply(remap)
df2 = df[df["label_remapped"] != 2]
df2

,label,score,label_remapped
1,false,0.4,0
2,false,0.4,0
3,false,0.2,0
4,false,0.2,0
5,true,0.8,1
...,...,...,...
967,false,0.4,0
968,false,0.2,0
969,false,0.2,0
970,false,0.2,0


In [134]:
# finding out the best threshold for f1 score
threshold = 0.0
all_preds2 = [x for x, condition in zip(all_preds, df["label_remapped"] != 2) if condition]
while threshold <= 1.0:
  def thr(value, threshold):
    if value >= threshold:
      return 1
    return 0

  predicted_labels = [thr(value, threshold) for value in all_preds2]
  f1 = f1_score(df2["label_remapped"], predicted_labels)
  print("Threshold: ", threshold, "; f1: ", f1)
  threshold += 0.05

Threshold:  0.0 ; f1:  0.3455083909180652
Threshold:  0.05 ; f1:  0.7578347578347578
Threshold:  0.1 ; f1:  0.7596439169139466
Threshold:  0.15000000000000002 ; f1:  0.7454545454545455
Threshold:  0.2 ; f1:  0.7461773700305812
Threshold:  0.25 ; f1:  0.7407407407407408
Threshold:  0.3 ; f1:  0.7414330218068536
Threshold:  0.35 ; f1:  0.7335423197492164
Threshold:  0.39999999999999997 ; f1:  0.7255520504731862
Threshold:  0.44999999999999996 ; f1:  0.7238095238095239
Threshold:  0.49999999999999994 ; f1:  0.7202572347266882
Threshold:  0.5499999999999999 ; f1:  0.7142857142857143
Threshold:  0.6 ; f1:  0.7166123778501629
Threshold:  0.65 ; f1:  0.712871287128713
Threshold:  0.7000000000000001 ; f1:  0.7152317880794702
Threshold:  0.7500000000000001 ; f1:  0.7046979865771812
Threshold:  0.8000000000000002 ; f1:  0.6983050847457626
Threshold:  0.8500000000000002 ; f1:  0.6917808219178082
Threshold:  0.9000000000000002 ; f1:  0.6805555555555557
Threshold:  0.9500000000000003 ; f1:  0.65480